# Import Packages

In [35]:
import mysql.connector
from mysql.connector import errorcode
import config
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import scipy.stats as stats
import seaborn as sns

# Connect to MySQL

In [3]:
dbname = 'nba'
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    db = dbname
)
cursor = cnx.cursor(buffered = True)

# Get box scores

In [4]:
select_statement = """SELECT * FROM nba.box_score;"""
cursor.execute(select_statement)
nba_data = cursor.fetchall()

In [5]:
nba_df = pd.DataFrame(nba_data)
field_names = [i[0] for i in cursor.description]
nba_df.columns = field_names

In [6]:
nba_df_played = nba_df[nba_df['MIN'].notnull()]

In [7]:
nba_df_played.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57287 entries, 3 to 67817
Data columns (total 34 columns):
GAME_ID              57287 non-null object
TEAM_ID              57287 non-null float64
TEAM_ABBREVIATION    57287 non-null object
TEAM_CITY            57287 non-null object
PLAYER_ID            57287 non-null float64
PLAYER_NAME          57287 non-null object
START_POSITION       57287 non-null object
COMMENT              57287 non-null object
MIN                  57287 non-null object
FGM                  57287 non-null float64
FGA                  57287 non-null float64
FG_PCT               57287 non-null float64
FG3M                 57287 non-null float64
FG3A                 57287 non-null float64
FG3_PCT              57287 non-null float64
FTM                  57287 non-null float64
FTA                  57287 non-null float64
FT_PCT               57287 non-null float64
OREB                 57287 non-null float64
DREB                 57287 non-null float64
REB               

In [9]:
nba_df_played.shape

(57287, 34)

# Create second variable

In [12]:
nba_df_played.reset_index(drop = True, inplace = True)
nba_df_played['SEC'] = 0
for i in range(0, len(nba_df_played)):
    minutes = nba_df_played.loc[i, 'MIN']
    nba_df_played.at[i, 'SEC'] = int(minutes.split(':')[0])*60 + int(minutes.split(':')[1])

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


# Get Trailing Averages for Last 3 Games

In [13]:
# Get average from last trail_len
def get_trailing_avg(df, col_name, trail_len):
    new_col = col_name + '_TRAIL' + str(trail_len)
    df[new_col] = None
    for i in range(0, trail_len):
        df.at[i, new_col] = 'game' + str(i + 1)
    for index in range(trail_len, len(df)):
        df.at[index, new_col] = np.mean(df.loc[index-trail_len:index-1, col_name])

In [14]:
nba_trailing = []
for season in set(nba_df_played['SEASON_ID']):
    for player in set(nba_df_played['PLAYER_ID']):
        games_played = nba_df_played[(nba_df_played['PLAYER_ID'] == player) & 
                                     (nba_df_played['SEASON_ID'] == season)]
        games_played = games_played.sort_values(by = ['GAME_ID'])
        games_played.reset_index(drop = True, inplace = True)
                      
        # Group 1: Field Goals
        get_trailing_avg(games_played, 'FGM', 3)
        get_trailing_avg(games_played, 'FGA', 3)
        get_trailing_avg(games_played, 'FG_PCT', 3)
        
        # Group 2: Three-Pointers
        get_trailing_avg(games_played, 'FG3M', 3)
        get_trailing_avg(games_played, 'FG3A', 3)
        get_trailing_avg(games_played, 'FG3_PCT', 3)

        # Group 3: Free Throws
        get_trailing_avg(games_played, 'FTM', 3)
        get_trailing_avg(games_played, 'FTA', 3)
        get_trailing_avg(games_played, 'FT_PCT', 3)

        # Group 4: Rebounds
        get_trailing_avg(games_played, 'OREB', 3)
        get_trailing_avg(games_played, 'DREB', 3)
        get_trailing_avg(games_played, 'REB', 3)
        
        # Group 5: Miscellaneous
        get_trailing_avg(games_played, 'PTS', 3)
        get_trailing_avg(games_played, 'STL', 3)
        get_trailing_avg(games_played, 'AST', 3)            
        get_trailing_avg(games_played, 'BLK', 3)
        get_trailing_avg(games_played, 'TOS', 3)
        get_trailing_avg(games_played, 'doubles', 3)
        get_trailing_avg(games_played, 'SEC', 3)
                      
        nba_trailing.append(games_played)

In [15]:
nba_trailing_df = pd.concat(nba_trailing)

In [16]:
nba_trailing_played = nba_trailing_df[nba_trailing_df['MIN'].notnull()]

In [17]:
nba_trailing_played.columns

Index(['GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_CITY', 'PLAYER_ID',
       'PLAYER_NAME', 'START_POSITION', 'COMMENT', 'MIN', 'FGM', 'FGA',
       'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB',
       'DREB', 'REB', 'AST', 'STL', 'BLK', 'PF', 'PTS', 'PLUS_MINUS', 'pk',
       'TOS', 'SEASON_ID', 'DK_PTS', 'DD', 'TD', 'doubles', 'SEC',
       'FGM_TRAIL3', 'FGA_TRAIL3', 'FG_PCT_TRAIL3', 'FG3M_TRAIL3',
       'FG3A_TRAIL3', 'FG3_PCT_TRAIL3', 'FTM_TRAIL3', 'FTA_TRAIL3',
       'FT_PCT_TRAIL3', 'OREB_TRAIL3', 'DREB_TRAIL3', 'REB_TRAIL3',
       'PTS_TRAIL3', 'STL_TRAIL3', 'AST_TRAIL3', 'BLK_TRAIL3', 'TOS_TRAIL3',
       'doubles_TRAIL3', 'SEC_TRAIL3'],
      dtype='object')

In [41]:
nba_trailing_played.to_csv('NBA_box_scores_trailing3.csv')